In [1]:
import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import re
import os
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('C:/Users/kantw/Downloads/Training Data.csv')
data

,user_id,aov,category
0,37327,29128.0,Phones
1,37327,354.0,Fashion
2,37327,460.0,Home Decor
3,37327,761.0,Fashion
4,41260,16658.0,Phones
...,...,...,...
257402,29237,689.0,Fashion
257403,15197,2195.0,Groceries
257404,2114,559.0,Back to School
257405,35741,18451.0,Phones


In [3]:
userid = list(data['user_id'])
aov = list(data['aov'])
category = list(data['category'])
d = {}
for i in range(len(userid)):
    if d.get(userid[i]):
        if d[userid[i]].get(category[i]):
            d[userid[i]][category[i]].append(aov[i])
        else:
            d[userid[i]][category[i]] = [aov[i]]
    else:
        d[userid[i]] = {category[i]:[aov[i]]}
d

{37327: {'Phones': [29128.0],
  'Fashion': [354.0, 761.0],
  'Home Decor': [460.0]},
 41260: {'Phones': [16658.0], 'Home Decor': [263.0], 'Fashion': [988.0]},
 19360: {'Phones': [10214.0, 16761.0, 6382.0, 12368.0],
  'Consumer Durables': [21026.0],
  'Back to School': [259.0],
  'Toys': [519.0, 690.0],
  'Fashion': [541.0, 566.0],
  'Home Decor': [580.0],
  'Books': [401.0],
  'Groceries': [1521.0]},
 39634: {'Home Decor': [169.0, 105.0, 134.0],
  'Fitness': [8770.0],
  'Board Games': [412.0],
  'Back to School': [252.0],
  'Phones': [13695.0],
  'Fashion': [684.0, 269.0],
  'Toys': [794.0],
  'Consumer Durables': [34148.0],
  'Painiting Supplies': [210.0]},
 25977: {'Gaming': [723.0],
  'Home Decor': [242.0],
  'Fashion': [1019.0, 857.0, 966.0, 582.0],
  'Consumer Durables': [36300.0],
  'Phones': [27916.0, 17181.0],
  'Groceries': [2650.0]},
 9818: {'Phones': [10194.0, 6743.0, 25799.0, 14321.0, 5741.0],
  'TVs': [57650.0],
  'Groceries': [1230.0],
  'Gaming': [961.0, 979.0, 138.0],
 

In [4]:
ndata = []
for i in d.keys():
    for j in d[i].keys():
        ndata.append([i,j,sum(d[i][j])])
ndata   

[[37327, 'Phones', 29128.0],
 [37327, 'Fashion', 1115.0],
 [37327, 'Home Decor', 460.0],
 [41260, 'Phones', 16658.0],
 [41260, 'Home Decor', 263.0],
 [41260, 'Fashion', 988.0],
 [19360, 'Phones', 45725.0],
 [19360, 'Consumer Durables', 21026.0],
 [19360, 'Back to School', 259.0],
 [19360, 'Toys', 1209.0],
 [19360, 'Fashion', 1107.0],
 [19360, 'Home Decor', 580.0],
 [19360, 'Books', 401.0],
 [19360, 'Groceries', 1521.0],
 [39634, 'Home Decor', 408.0],
 [39634, 'Fitness', 8770.0],
 [39634, 'Board Games', 412.0],
 [39634, 'Back to School', 252.0],
 [39634, 'Phones', 13695.0],
 [39634, 'Fashion', 953.0],
 [39634, 'Toys', 794.0],
 [39634, 'Consumer Durables', 34148.0],
 [39634, 'Painiting Supplies', 210.0],
 [25977, 'Gaming', 723.0],
 [25977, 'Home Decor', 242.0],
 [25977, 'Fashion', 3424.0],
 [25977, 'Consumer Durables', 36300.0],
 [25977, 'Phones', 45097.0],
 [25977, 'Groceries', 2650.0],
 [9818, 'Phones', 62798.0],
 [9818, 'TVs', 57650.0],
 [9818, 'Groceries', 1230.0],
 [9818, 'Gaming', 

In [5]:
mdata = pd.DataFrame(ndata, columns = ['user_id', 'category','aov'])
mdata

,user_id,category,aov
0,37327,Phones,29128.0
1,37327,Fashion,1115.0
2,37327,Home Decor,460.0
3,41260,Phones,16658.0
4,41260,Home Decor,263.0
...,...,...,...
159864,50033,Back to School,960.0
159865,28975,Back to School,495.0
159866,50030,Groceries,184.0
159867,50066,Groceries,2840.0


In [6]:
maindata = mdata.pivot_table(index=["user_id"],columns=["category"],values="aov")
maindata

category,Back to School,Beauty Products,Board Games,Books,Consumer Durables,Ereaders,Fashion,Fitness,Gaming,Groceries,Home Decor,Kitchen cleaning Supplies,Laptops,Painiting Supplies,Pet Supplies,Phones,TVs,Toys
user_id,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,1416.0,12056.0,NaN,447.0,NaN,NaN,NaN,NaN,NaN,10216.0,NaN,424.0
4,355.0,NaN,NaN,NaN,NaN,10555.0,1243.0,NaN,NaN,2224.0,578.0,NaN,NaN,NaN,NaN,38786.0,NaN,NaN
5,552.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1128.0,NaN,NaN,NaN,NaN,9023.0,NaN,NaN
6,NaN,NaN,NaN,851.0,NaN,NaN,3200.0,574.0,1012.0,1464.0,NaN,NaN,NaN,NaN,NaN,73316.0,NaN,1197.0
7,455.0,NaN,NaN,NaN,NaN,NaN,1330.0,NaN,NaN,1885.0,NaN,NaN,NaN,NaN,NaN,32012.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18861.0,NaN,NaN
50078,NaN,NaN,NaN,806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,726.0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
maindata.replace(np.nan, 0, inplace=True)
maindata

category,Back to School,Beauty Products,Board Games,Books,Consumer Durables,Ereaders,Fashion,Fitness,Gaming,Groceries,Home Decor,Kitchen cleaning Supplies,Laptops,Painiting Supplies,Pet Supplies,Phones,TVs,Toys
user_id,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,1416.0,12056.0,0.0,447.0,0.0,0.0,0.0,0.0,0.0,10216.0,0.0,424.0
4,355.0,0.0,0.0,0.0,0.0,10555.0,1243.0,0.0,0.0,2224.0,578.0,0.0,0.0,0.0,0.0,38786.0,0.0,0.0
5,552.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1128.0,0.0,0.0,0.0,0.0,9023.0,0.0,0.0
6,0.0,0.0,0.0,851.0,0.0,0.0,3200.0,574.0,1012.0,1464.0,0.0,0.0,0.0,0.0,0.0,73316.0,0.0,1197.0
7,455.0,0.0,0.0,0.0,0.0,0.0,1330.0,0.0,0.0,1885.0,0.0,0.0,0.0,0.0,0.0,32012.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18861.0,0.0,0.0
50078,0.0,0.0,0.0,806.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,726.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
maindata['Phones'][19360]
maindata.index,maindata.columns

(Int64Index([    1,     4,     5,     6,     7,     9,    11,    12,    14,
                16,
             ...
             50030, 50033, 50035, 50062, 50066, 50075, 50078, 50079, 50080,
             50082],
            dtype='int64', name='user_id', length=29972),
 Index(['Back to School', 'Beauty Products', 'Board Games', 'Books',
        'Consumer Durables', 'Ereaders', 'Fashion', 'Fitness', 'Gaming',
        'Groceries', 'Home Decor', 'Kitchen cleaning Supplies', 'Laptops',
        'Painiting Supplies', 'Pet Supplies', 'Phones', 'TVs', 'Toys'],
       dtype='object', name='category'))

In [9]:
similarity_data = cosine_similarity(maindata.T)

In [10]:
similarity_data

array([[1.        , 0.13359043, 0.14358229, 0.19731145, 0.34073109,
        0.13082859, 0.41848518, 0.1340146 , 0.2534166 , 0.2562718 ,
        0.34363137, 0.14897839, 0.13900156, 0.14969327, 0.14711238,
        0.47442225, 0.13613996, 0.30086371],
       [0.13359043, 1.        , 0.06956384, 0.10187623, 0.1839967 ,
        0.0702418 , 0.21768898, 0.07668449, 0.13688122, 0.14036251,
        0.18809346, 0.07795311, 0.0738547 , 0.06673454, 0.06973832,
        0.25155364, 0.07219448, 0.16188541],
       [0.14358229, 0.06956384, 1.        , 0.09993401, 0.19111544,
        0.08141806, 0.22874208, 0.07407173, 0.14354362, 0.15294845,
        0.18644585, 0.08233254, 0.0774244 , 0.08631441, 0.08127313,
        0.26519193, 0.07482867, 0.1749421 ],
       [0.19731145, 0.10187623, 0.09993401, 1.        , 0.25767466,
        0.10475497, 0.31688944, 0.09855121, 0.18458163, 0.19355601,
        0.26044921, 0.10291669, 0.10613974, 0.1036944 , 0.11721039,
        0.35970979, 0.10841706, 0.22308636],
    

In [11]:
similarity_data_df = pd.DataFrame(similarity_data,index=maindata.columns,columns=maindata.columns)

In [12]:
similarity_data_df

category,Back to School,Beauty Products,Board Games,Books,Consumer Durables,Ereaders,Fashion,Fitness,Gaming,Groceries,Home Decor,Kitchen cleaning Supplies,Laptops,Painiting Supplies,Pet Supplies,Phones,TVs,Toys
category,,,,,,,,,,,,,,,,,,
Back to School,1.000000,0.133590,0.143582,0.197311,0.340731,0.130829,0.418485,0.134015,0.253417,0.256272,0.343631,0.148978,0.139002,0.149693,0.147112,0.474422,0.136140,0.300864
Beauty Products,0.133590,1.000000,0.069564,0.101876,0.183997,0.070242,0.217689,0.076684,0.136881,0.140363,0.188093,0.077953,0.073855,0.066735,0.069738,0.251554,0.072194,0.161885
Board Games,0.143582,0.069564,1.000000,0.099934,0.191115,0.081418,0.228742,0.074072,0.143544,0.152948,0.186446,0.082333,0.077424,0.086314,0.081273,0.265192,0.074829,0.174942
Books,0.197311,0.101876,0.099934,1.000000,0.257675,0.104755,0.316889,0.098551,0.184582,0.193556,0.260449,0.102917,0.106140,0.103694,0.117210,0.359710,0.108417,0.223086
Consumer Durables,0.340731,0.183997,0.191115,0.257675,1.000000,0.189304,0.559991,0.175596,0.339599,0.348122,0.463389,0.200633,0.236179,0.191890,0.194532,0.637349,0.227827,0.418399
Ereaders,0.130829,0.070242,0.081418,0.104755,0.189304,1.000000,0.230510,0.071008,0.144765,0.146773,0.189896,0.079523,0.076480,0.085521,0.077809,0.266468,0.066882,0.174483
Fashion,0.418485,0.217689,0.228742,0.316889,0.559991,0.230510,1.000000,0.213811,0.404372,0.425114,0.560539,0.239575,0.224920,0.227052,0.244773,0.769177,0.218866,0.497690
Fitness,0.134015,0.076684,0.074072,0.098551,0.175596,0.071008,0.213811,1.000000,0.131516,0.141422,0.177541,0.089734,0.072231,0.081419,0.083533,0.251548,0.075700,0.168520
Gaming,0.253417,0.136881,0.143544,0.184582,0.339599,0.144765,0.404372,0.131516,1.000000,0.254546,0.337481,0.137836,0.139381,0.134865,0.151357,0.465064,0.135022,0.308225


In [13]:
columns = list(maindata.columns)
def get_simi(pname):
    similarity_score = similarity_data_df[pname]
    scorel = list(similarity_score)
    nsd = {}
    for i in range(len(scorel)):
        nsd[scorel[i]] = i
    scorel.sort(reverse=True)
    for i in range(1,4):
        nml.append(columns[nsd[scorel[i]]]) 

In [14]:
ans = {}
from collections import Counter
for j in range(50082):
    nml = []
    if d.get(j):
        for i in d[j].keys():
            get_simi(i)
        p = dict(Counter(nml))
        p = dict(sorted(p.items(), key=lambda item: item[1],reverse=True)).keys()
        ans[j] = list(p)[0:3]
ans

{1: ['Phones', 'Home Decor', 'Fashion'],
 4: ['Fashion', 'Phones', 'Consumer Durables'],
 5: ['Fashion', 'Consumer Durables', 'Home Decor'],
 6: ['Phones', 'Fashion', 'Home Decor'],
 7: ['Fashion', 'Consumer Durables', 'Home Decor'],
 9: ['Phones', 'Fashion', 'Home Decor'],
 11: ['Phones', 'Fashion', 'Home Decor'],
 12: ['Fashion', 'Consumer Durables', 'Home Decor'],
 14: ['Home Decor', 'Phones', 'Fashion'],
 16: ['Fashion', 'Home Decor', 'Phones'],
 19: ['Phones', 'Fashion', 'Home Decor'],
 21: ['Consumer Durables', 'Fashion', 'Home Decor'],
 25: ['Phones', 'Fashion', 'Consumer Durables'],
 26: ['Phones', 'Fashion', 'Home Decor'],
 27: ['Fashion', 'Home Decor', 'Phones'],
 28: ['Consumer Durables', 'Fashion', 'Home Decor'],
 29: ['Fashion', 'Consumer Durables', 'Home Decor'],
 30: ['Phones', 'Home Decor', 'Fashion'],
 33: ['Home Decor', 'Phones', 'Fashion'],
 34: ['Fashion', 'Consumer Durables', 'Phones'],
 35: ['Phones', 'Fashion', 'Consumer Durables'],
 39: ['Fashion', 'Consumer Dur

In [15]:
print(ans)

{1: ['Phones', 'Home Decor', 'Fashion'], 4: ['Fashion', 'Phones', 'Consumer Durables'], 5: ['Fashion', 'Consumer Durables', 'Home Decor'], 6: ['Phones', 'Fashion', 'Home Decor'], 7: ['Fashion', 'Consumer Durables', 'Home Decor'], 9: ['Phones', 'Fashion', 'Home Decor'], 11: ['Phones', 'Fashion', 'Home Decor'], 12: ['Fashion', 'Consumer Durables', 'Home Decor'], 14: ['Home Decor', 'Phones', 'Fashion'], 16: ['Fashion', 'Home Decor', 'Phones'], 19: ['Phones', 'Fashion', 'Home Decor'], 21: ['Consumer Durables', 'Fashion', 'Home Decor'], 25: ['Phones', 'Fashion', 'Consumer Durables'], 26: ['Phones', 'Fashion', 'Home Decor'], 27: ['Fashion', 'Home Decor', 'Phones'], 28: ['Consumer Durables', 'Fashion', 'Home Decor'], 29: ['Fashion', 'Consumer Durables', 'Home Decor'], 30: ['Phones', 'Home Decor', 'Fashion'], 33: ['Home Decor', 'Phones', 'Fashion'], 34: ['Fashion', 'Consumer Durables', 'Phones'], 35: ['Phones', 'Fashion', 'Consumer Durables'], 39: ['Fashion', 'Consumer Durables', 'Phones'], 41

In [16]:
tdata = pd.read_csv('C:/Users/kantw/Downloads/Test Data.csv')
tdata

,Unnamed: 0,user_id,aov,category
0,0,43323,19115.0,Phones
1,1,43323,29309.0,Phones
2,2,43323,15293.0,Phones
3,3,43323,23548.0,Phones
4,4,43323,751.0,Fashion
...,...,...,...,...
172549,86137,12403,1841.0,Groceries
172550,86138,31036,424.0,Back to School
172551,86139,12652,617.0,Fashion
172552,86140,35726,634.0,Kitchen cleaning Supplies


In [17]:
userid = list(tdata['user_id'])
aov = list(tdata['aov'])
category = list(tdata['category'])
td = {}
for i in range(len(userid)):
    if td.get(userid[i]):
        if td[userid[i]].get(category[i]):
            td[userid[i]][category[i]].append(aov[i])
        else:
            td[userid[i]][category[i]] = [aov[i]]
    else:
        td[userid[i]] = {category[i]:[aov[i]]}
td

{43323: {'Phones': [19115.0, 29309.0, 15293.0, 23548.0],
  'Fashion': [751.0, 1017.0, 610.0, 490.0, 867.0],
  'Consumer Durables': [41480.0],
  'Back to School': [419.0, 461.0],
  'Books': [642.0]},
 4269: {'Ereaders': [15659.0],
  'Fashion': [669.0, 717.0, 869.0],
  'Phones': [26322.0],
  'Home Decor': [114.0],
  'Back to School': [390.0]},
 21731: {'Toys': [111.0, 1181.0],
  'Home Decor': [432.0],
  'Groceries': [461.0, 2039.0],
  'Phones': [24241.0, 20185.0, 19150.0, 6768.0],
  'Board Games': [641.0],
  'Fashion': [536.0, 747.0],
  'Consumer Durables': [31367.0],
  'Gaming': [826.0]},
 14422: {'Consumer Durables': [54107.0],
  'Phones': [11847.0, 16665.0],
  'TVs': [15710.0],
  'Gaming': [1497.0],
  'Back to School': [564.0],
  'Home Decor': [437.0],
  'Fashion': [678.0, 630.0],
  'Beauty Products': [395.0],
  'Kitchen cleaning Supplies': [512.0]},
 12862: {'Consumer Durables': [42299.0],
  'Fashion': [727.0, 730.0, 835.0],
  'Toys': [563.0, 671.0],
  'Phones': [11031.0, 8353.0, 122

In [32]:
ans = {}
from collections import Counter
for j in range(1725530):
    nml = []
    if td.get(j):
        for i in td[j].keys():
            get_simi(i)
        p = dict(Counter(nml))
        p = dict(sorted(p.items(), key=lambda item: item[1],reverse=True)).keys()
        ans[j] = list(p)[0:3]
ans

{2: ['Home Decor', 'Phones', 'Fashion'],
 3: ['Home Decor', 'Fashion', 'Phones'],
 8: ['Phones', 'Fashion', 'Home Decor'],
 10: ['Home Decor', 'Fashion', 'Consumer Durables'],
 13: ['Fashion', 'Phones', 'Home Decor'],
 15: ['Fashion', 'Consumer Durables', 'Phones'],
 17: ['Phones', 'Home Decor', 'Fashion'],
 18: ['Consumer Durables', 'Fashion', 'Home Decor'],
 20: ['Fashion', 'Consumer Durables', 'Home Decor'],
 22: ['Fashion', 'Home Decor', 'Phones'],
 23: ['Home Decor', 'Fashion', 'Consumer Durables'],
 24: ['Phones', 'Fashion', 'Home Decor'],
 31: ['Fashion', 'Phones', 'Home Decor'],
 32: ['Phones', 'Fashion', 'Home Decor'],
 36: ['Fashion', 'Consumer Durables', 'Phones'],
 37: ['Fashion', 'Phones', 'Home Decor'],
 38: ['Phones', 'Fashion', 'Consumer Durables'],
 40: ['Home Decor', 'Fashion', 'Consumer Durables'],
 44: ['Consumer Durables', 'Fashion', 'Home Decor'],
 46: ['Fashion', 'Consumer Durables', 'Phones'],
 49: ['Fashion', 'Home Decor', 'Phones'],
 54: ['Phones', 'Home Decor

In [33]:
ans[4229]

['Phones', 'Fashion', 'Home Decor']

In [34]:
print(ans)

{2: ['Home Decor', 'Phones', 'Fashion'], 3: ['Home Decor', 'Fashion', 'Phones'], 8: ['Phones', 'Fashion', 'Home Decor'], 10: ['Home Decor', 'Fashion', 'Consumer Durables'], 13: ['Fashion', 'Phones', 'Home Decor'], 15: ['Fashion', 'Consumer Durables', 'Phones'], 17: ['Phones', 'Home Decor', 'Fashion'], 18: ['Consumer Durables', 'Fashion', 'Home Decor'], 20: ['Fashion', 'Consumer Durables', 'Home Decor'], 22: ['Fashion', 'Home Decor', 'Phones'], 23: ['Home Decor', 'Fashion', 'Consumer Durables'], 24: ['Phones', 'Fashion', 'Home Decor'], 31: ['Fashion', 'Phones', 'Home Decor'], 32: ['Phones', 'Fashion', 'Home Decor'], 36: ['Fashion', 'Consumer Durables', 'Phones'], 37: ['Fashion', 'Phones', 'Home Decor'], 38: ['Phones', 'Fashion', 'Consumer Durables'], 40: ['Home Decor', 'Fashion', 'Consumer Durables'], 44: ['Consumer Durables', 'Fashion', 'Home Decor'], 46: ['Fashion', 'Consumer Durables', 'Phones'], 49: ['Fashion', 'Home Decor', 'Phones'], 54: ['Phones', 'Home Decor', 'Fashion'], 61: ['

In [35]:
ans[21731]

['Phones', 'Fashion', 'Consumer Durables']

In [36]:
mans = []
for i in ans.keys():
    mans.append([i,", ".join(ans[i])])
mans

[[2, 'Home Decor, Phones, Fashion'],
 [3, 'Home Decor, Fashion, Phones'],
 [8, 'Phones, Fashion, Home Decor'],
 [10, 'Home Decor, Fashion, Consumer Durables'],
 [13, 'Fashion, Phones, Home Decor'],
 [15, 'Fashion, Consumer Durables, Phones'],
 [17, 'Phones, Home Decor, Fashion'],
 [18, 'Consumer Durables, Fashion, Home Decor'],
 [20, 'Fashion, Consumer Durables, Home Decor'],
 [22, 'Fashion, Home Decor, Phones'],
 [23, 'Home Decor, Fashion, Consumer Durables'],
 [24, 'Phones, Fashion, Home Decor'],
 [31, 'Fashion, Phones, Home Decor'],
 [32, 'Phones, Fashion, Home Decor'],
 [36, 'Fashion, Consumer Durables, Phones'],
 [37, 'Fashion, Phones, Home Decor'],
 [38, 'Phones, Fashion, Consumer Durables'],
 [40, 'Home Decor, Fashion, Consumer Durables'],
 [44, 'Consumer Durables, Fashion, Home Decor'],
 [46, 'Fashion, Consumer Durables, Phones'],
 [49, 'Fashion, Home Decor, Phones'],
 [54, 'Phones, Home Decor, Fashion'],
 [61, 'Phones, Fashion, Home Decor'],
 [64, 'Phones, Fashion, Home Decor'

In [37]:
ndf = pd.DataFrame(mans,columns = ['user_id', 'pred3'])
ndf.set_index('user_id', inplace=True)
ndf

,pred3
user_id,
2,"Home Decor, Phones, Fashion"
3,"Home Decor, Fashion, Phones"
8,"Phones, Fashion, Home Decor"
10,"Home Decor, Fashion, Consumer Durables"
13,"Fashion, Phones, Home Decor"
...,...
49998,"Consumer Durables, Fashion, Home Decor"
50037,"Phones, Fashion, Consumer Durables"
50050,"Phones, Fashion, Home Decor"


In [38]:
ndf.to_csv('C:/Users/kantw/Downloads/resultdata5.csv')

In [254]:
l = [1,1,2,4,3,3,6,6,3,2]
print(Counter(l).keys(),Counter(l))

dict_keys([1, 2, 4, 3, 6]) Counter({3: 3, 1: 2, 2: 2, 6: 2, 4: 1})


In [263]:
p = dict(Counter(l))
p = dict(sorted(p.items(), key=lambda item: item[1],reverse=True))

In [264]:
p

{3: 3, 1: 2, 2: 2, 6: 2, 4: 1}

In [265]:
for i in p:
    print(i)

3
1
2
6
4


In [268]:
p.keys()

dict_keys([3, 1, 2, 6, 4])

In [1]:
allans = []
for b in range(1,21):
    rl = []
    for i in range(1,b+1):
        rl.append(i)
    
    d = {}
    def getAns(rl):
        if len(rl) == 0:
            return []
        elif len(rl) == 1:
            return rl
        else:
            l = [rl]
            for i in range(len(rl)):
                t = tuple(rl[i+1:])
                if d.get(t):
                    x = d[t]
                else:
                    x = getAns(rl[i+1:])
                    d[t] = x
                
                for k in x:
                    l.append([rl[:i+1],k])
            return l
    l = getAns(rl)
    lnth = len(l)
    if lnth == 1:
        nl = [[sum(l)]]
    else:
        nl = [[sum(l[0])]]
    
    for i in range(1,lnth):
        m = l[i]
        nll = []
        while type(m) is list:
            nll.append(sum(m[0]))
            if len(m) > 1:
                if type(m[1]) is list:
                    m = m[1]
                    if type(m[0]) is list:
                        pass
                    else:
                        nll.append(sum(m))
                        break
                else:
                    nll.append(m[1])
                    break
        nl.append(nll)
    ans = 0
    for i in nl:
        Flag = True
        for j in range(len(i)-1):
            if i[j]%2 == i[j+1]%2:
                Flag = False
                break
        if Flag:
            ans += 1
    allans.append(ans)
print(allans)

[1, 2, 2, 3, 6, 10, 12, 21, 29, 48, 69, 117, 157, 266, 370, 623, 862, 1458, 2004, 3385]
